In [1]:
from IPython.display import Image

https://zhuanlan.zhihu.com/p/12871616401

- 为每个actor分配其对应的critic/reward/ref，并启动每个分组的训练：
    - https://github.com/OpenRLHF/OpenRLHF/blob/bb46342711a203c457df2fbca5967fd0549557e0/openrlhf/trainer/ray/launcher.py#L278-L299
- 为每个actor分配对应的vllm_engine，并使用vllm_engine进行推理：
    - https://github.com/OpenRLHF/OpenRLHF/blob/bb46342711a203c457df2fbca5967fd0549557e0/openrlhf/trainer/ppo_utils/experience_maker.py#L627

In [3]:
Image(url='https://pica.zhimg.com/v2-c46a2e47aa48f3c0a42eecc5003e28ee_1440w.jpg', width=600)

- 在这个例子中，4类模型分开部署在node0和node1上。以Actor为例，它分布在“node0的gpu0/1 + node1的gpu0/1”上。这一点是由Ray实现的：我们自己定制化资源分配的方案，进而管控模型的分配方式
    - 而当实际训练时，我们还可进一步引入Deepspeed zero做优化：以Actor为例，上图中的4个Actor构成zero中的数据并行组（world_size = 4），根据zero的配置，我们可以在这4张卡间做optimizer(1)/gradients(1)/weights(2)的切片
- 部署vllm_engines
    - 对于Actor模型，在收集exp阶段我们可以采用vllm之类的框架加速(prompt, responses)的生成。在这个例子中：
        - 1个vllm_engine维护着一个vllm实例，每个vllm实例下维护一个完整的Actor模型，这里我们还假设一个vllm实例按tp_size = 2的方法切割模型。
        - 在node2中，共有4个vllm_engines（也即4个vllm实例），这种分配方式是通过Ray实现的。而每个vllm实例内的分布式推理则是由vllm自己管控。
- Actor与vllm_engines之间的通讯
    - 首先，对ds_rank0 + all_vllm_ranks创建一个通讯组。在本例中:
        - node0/gpu0上的actor是ds_rank0，node2中所有的gpu构成all_vllm_ranks。
    - 我们就是把这两者纳入一个通讯组内，这个通讯组的world_size = 9。如果我们多一台node3来做vllm_engines，那么这个通讯组的world_size = 17，以此类推。
        - 假设我们使用ds_zero1/2，则ds_rank0上维护的是完整的actor权重，我们把ds_rank0上的权重broadcast到每一个vllm_rank，如有设置tp，vllm会自动帮我们完整接下来的模型切割。
        - 假设我们使用ds_zero3，则ds_rank0上只维护部分actor权重，那么：
            - ds_rank0先从ds_actor组内all gather回完整的模型权重
            - 再将完整的模型权重brocast给每一个vllm_rank

In [4]:
Image(url='https://pic2.zhimg.com/v2-b91c1b4dd04d93e8b06674f47099304f_1440w.jpg', width=600)